<a href="https://colab.research.google.com/github/Patasheva/Projet_final/blob/main/Qwen2_5_VL_3B_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers opencv-python
!pip install torch torchvision torchaudio einops timm pillow
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/accelerate
!pip install huggingface_hub
!pip install sentencepiece bitsandbytes protobuf decord numpy
!pip install qwen-vl-utils[decord]==0.0.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

video_path = "/content/drive/MyDrive/Personas_project/Data/videos/video_003.mp4"
print("🧭 Chemin existe ? ", os.path.exists(video_path))

🧭 Chemin existe ?  True


Étape 1 : Charger la vidéo et extraire une image

In [ ]:
import cv2
import os

# Charger la vidéo
video_path = "/content/drive/MyDrive/Personas_project/Data/videos/video_003.mp4"
cap = cv2.VideoCapture(video_path) # Ouvre le fichier vidéo pour lire ses images (frames) une par une.

# Obtenir nombre de frames et FPS
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Récupère le nombre total de frames dans la vidéo
fps = cap.get(cv2.CAP_PROP_FPS) # Récupère le nombre d’images par seconde (Frames Per Second)
middle_frame = total_frames // 2 # Calcule la frame du milieu
print(f"Total frames: {total_frames}, FPS: {fps}, Middle frame: {middle_frame}")

Total frames: 1035, FPS: 25.0, Middle frame: 517


In [ ]:
# Se déplacer au milieu
cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame) # Chemin vers la frame du milieu
ret, frame = cap.read() # ret (True ou False) - il dit si la lecture a réussi # frame contient l’image (sous forme de matrice NumPy)

# Sauvegarder l'image extraite
if ret:
    img_path = "/content/drive/MyDrive/Personas_project/Data/images/middel_frame_003.jpg"
    cv2.imwrite(img_path, frame)
    print(f"✅ Image extraite à {middle_frame}e frame")
else:
    print("❌ Impossible d'extraire l'image")

cap.release()

✅ Image extraite à 517e frame


Étape 2 : Charger le modèle Qwen2.5-VL

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from PIL import Image

# Charger le modèle

model_id = "Qwen/Qwen2.5-VL-3B-Instruct"
processor = AutoProcessor.from_pretrained(model_id)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_id,
                                                           torch_dtype=torch.bfloat16,
                                                           device_map="auto",)
model = model.to(device) # Placer le modèle sur ce device

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

 Étape 3: Interroger le modèle

In [ ]:
prompt = """Act as a Product Analyst tasked with analyzing congratulatory videos uploaded to our platform,
            extracting key information to build accurate and actionable user personas.
            You are analyzing one image extracted from a video.
            Return your answer strictly in the following JSON format:


            - For each person, estimate gender and approximate age using defined categories.

            {
              "motivation": "personal | professional",
              "occasion": "birthday | wedding | engagement | birth | baptism | love message | mother's day | father's day | farewell | get well | fun video | graduation | work anniversary | promotion | new job | success | team celebration | product presentation | company presentation | conference | retirement | new year | christmas | valentine's day | halloween | easter | international women's day",
              "recording_location": "home | living room | kitchen | bedroom | garden | outdoor | party | office | meeting room | open space | stage | car",
              "number_of_people": "1 | 2 | 3 | 4 | 5 | >5",
              "people": [
                {
                  "gender": "female | male",
                  "age_category": "baby | child | teenager | young_adult | adult | senior"
                }
              ]
            }

            Rules:
            - Select **only one** option from the predefined list of categories. If uncertain, return 'other'.
            - `number_of_people` must reflect how many people are **clearly visible** in the image.
            - For each people, estimate their age **in years**, and classify them using the following age categories:
              - `"baby"`: 0–2 years
              - `"child"`: 3–12 years
              - `"teenager"`: 13–19 years
              - `"young_adult"`: 20–35 years
              - `"adult"`: 36–59 years
              - `"senior"`: 60 years and older
              - Return **only the category label** (e.g., `"teenager"`, `"adult"`, etc.) — do not return the age number.
              - Note: All age estimates must be in **years**, not months or any other unit.

           Only return valid JSON.
          """

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": img_path},
            {"type": "text", "text": prompt},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['{\n  "motivation": "personal",\n  "occasion": "birthday",\n  "recording_location": "living room",\n  "number_of_people": "1",\n  "people": [\n    {\n      "gender": "male",\n      "age_category": "young_adult"\n    }\n  ]\n}']
